# Python ile Şeffaflık Platformu Web Servis Kullanımı

## Kütüphaneler (Packages)

In [1]:
# Web servis üzerinden istek yapabilmesi için gerekli kütüphane 
import requests
# İstek sonrası elde edilen verinin düzenlenmesi ve işlenmesi için gerekli kütüphane
import pandas as pd

## İstek Başlıkları (Headers)
Şeffaflık Platformu Web Servis isteklerinde istek başlıkları ile birlikte kabul edilecek olan veri tipi (accept) ve kimlik doğrulama (x-ibm-client-id) bilgileri gönderilmelidir. Kimlik doğrulama için gerekli istemci taniticisi (clint id) şeffaflık platformu web servis kaydı yapılarak temin edilmelidir. 

[Şeffaflık Platformu Web Servis Kullanım İzni Nasıl Alınır?](https://nurisensoy.github.io/seffaflikplatformu/nedir)

In [2]:
headers = {
    'x-ibm-client-id': "Tanımlanmış_İstemci_Tanıcısı",
    'accept': "application/json"
}

## Piyasa Takas Fiyatı İsteği (Requests)
Piyasa Takas Fiyatı (PTF) sorgulamasının yapılabilmesi için ilgili url kullanılmalıdır. Yapılacak olan diğer sorgulamalar için gerekli url'lere [Şeffaflık Platformu Web Sitesinden](https://apiportal.epias.com.tr/exchange/tr/node/1207) ulaşılabilir.

In [3]:
ptf_url = "https://api.epias.com.tr/epias/exchange/transparency/market/day-ahead-mcp?startDate={}&endDate={}"
resp = requests.get(ptf_url.format("2019-01-01","2019-01-31"), headers=headers)
resp.raise_for_status()

## İstek Yanıtı (Response)
İsteğin başarılı olması durumunda veri  json formatında  elde edilmektedir. 

In [4]:
json_data = resp.json()
json_data

{'resultCode': '0',
 'resultDescription': 'success',
 'body': {'dayAheadMCPList': [{'date': '2019-01-01T00:00:00.000+0300',
    'price': 100.38},
   {'date': '2019-01-01T01:00:00.000+0300', 'price': 96.72},
   {'date': '2019-01-01T02:00:00.000+0300', 'price': 81.6},
   {'date': '2019-01-01T03:00:00.000+0300', 'price': 38.58},
   {'date': '2019-01-01T04:00:00.000+0300', 'price': 11.52},
   {'date': '2019-01-01T05:00:00.000+0300', 'price': 11.14},
   {'date': '2019-01-01T06:00:00.000+0300', 'price': 11.14},
   {'date': '2019-01-01T07:00:00.000+0300', 'price': 24.37},
   {'date': '2019-01-01T08:00:00.000+0300', 'price': 34.5},
   {'date': '2019-01-01T09:00:00.000+0300', 'price': 45.21},
   {'date': '2019-01-01T10:00:00.000+0300', 'price': 71.54},
   {'date': '2019-01-01T11:00:00.000+0300', 'price': 100.31},
   {'date': '2019-01-01T12:00:00.000+0300', 'price': 102.94},
   {'date': '2019-01-01T13:00:00.000+0300', 'price': 100.36},
   {'date': '2019-01-01T14:00:00.000+0300', 'price': 104.73}

## Verinin İşlenebilir Formata Getirilmesi (DataFrame)
Verinin okunabilirliğinin arttırılması ve veri üzerinde ggerekli düzenleme ve işlemlerin yapılabilmesi için pandas kütüphanesi kullanılabilir.

In [5]:
df = pd.DataFrame(json_data["body"]["dayAheadMCPList"])
df.head()

,date,price
0,2019-01-01T00:00:00.000+0300,100.38
1,2019-01-01T01:00:00.000+0300,96.72
2,2019-01-01T02:00:00.000+0300,81.60
3,2019-01-01T03:00:00.000+0300,38.58
4,2019-01-01T04:00:00.000+0300,11.52


Tarih ve saat bilgileri date kolonuna **apply** methodu uygulanarak elde edilebilir.

In [6]:
df["Tarih"] = df["date"].apply(lambda x : pd.to_datetime(x[:10]))
df["Saat"] = df["date"].apply(lambda x : int(x[11:13]))

In [7]:
df.head()

,date,price,Tarih,Saat
0,2019-01-01T00:00:00.000+0300,100.38,2019-01-01,0
1,2019-01-01T01:00:00.000+0300,96.72,2019-01-01,1
2,2019-01-01T02:00:00.000+0300,81.60,2019-01-01,2
3,2019-01-01T03:00:00.000+0300,38.58,2019-01-01,3
4,2019-01-01T04:00:00.000+0300,11.52,2019-01-01,4


In [8]:
df.drop(columns=["date"],inplace=True)

In [9]:
df.head()

,price,Tarih,Saat
0,100.38,2019-01-01,0
1,96.72,2019-01-01,1
2,81.60,2019-01-01,2
3,38.58,2019-01-01,3
4,11.52,2019-01-01,4


In [10]:
df.columns = ["PTF", "Tarih", "Saat"]
df = df[["Tarih","Saat","PTF"]]
df.head()

,Tarih,Saat,PTF
0,2019-01-01,0,100.38
1,2019-01-01,1,96.72
2,2019-01-01,2,81.60
3,2019-01-01,3,38.58
4,2019-01-01,4,11.52
